In [1]:
# this is my first shoot in loanPred III competition

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [17]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [3]:
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


### remove NAN values in train set

In [24]:
for col in train.columns:
    
    print(col,sum(train[col].isnull()))
    

Loan_ID 0
Gender 0
Married 0
Dependents 0
Education 0
Self_Employed 0
ApplicantIncome 0
CoapplicantIncome 0
LoanAmount 21
Loan_Amount_Term 0
Credit_History 0
Property_Area 0
Loan_Status 0


In [5]:
# replace NA LoanAmount with mean
train.loc[train.LoanAmount.isnull(),'LoanAmount']=np.mean(train.loc[train.LoanAmount.notnull(),
                                                                    'LoanAmount'] ) 

In [19]:
train[train.Dependents.isnull()]

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
102,LP001350,Male,Yes,NaN,Graduate,No,13650,0.0,NaN,360.0,1.0,Urban,Y
104,LP001357,Male,NaN,NaN,Graduate,No,3816,754.0,160.0,360.0,1.0,Urban,Y
120,LP001426,Male,Yes,NaN,Graduate,No,5667,2667.0,180.0,360.0,1.0,Rural,Y
226,LP001754,Male,Yes,NaN,Not Graduate,Yes,4735,0.0,138.0,360.0,1.0,Urban,N
228,LP001760,Male,NaN,NaN,Graduate,No,4758,0.0,158.0,480.0,1.0,Semiurban,Y
293,LP001945,Female,No,NaN,Graduate,No,5417,0.0,143.0,480.0,0.0,Urban,N
301,LP001972,Male,Yes,NaN,Not Graduate,No,2875,1750.0,105.0,360.0,1.0,Semiurban,Y
332,LP002100,Male,No,NaN,Graduate,No,2833,0.0,71.0,360.0,1.0,Urban,Y
335,LP002106,Male,Yes,NaN,Graduate,Yes,5503,4490.0,70.0,NaN,1.0,Semiurban,Y
346,LP002130,Male,Yes,NaN,Not Graduate,No,3523,3230.0,152.0,360.0,0.0,Rural,N


In [20]:
# not married and Female have not independent 
train.loc[(train.Dependents.isnull()) & (  train.Married=='No'),'Dependents']=str(0)
train.loc[(train.Dependents.isnull()) & (  train.Gender=='Female'),'Dependents']=str(0)
# dependent nan replace with 0
train.loc[(train.Dependents.isnull())]=str(0)

In [23]:

train.loc[train.Gender.isnull(),'Gender']='Male'
train.loc[train.Married.isnull(),'Married']='Yes'
train.loc[train.Credit_History.isnull(),'Credit_History']=1.0
train.loc[train.Loan_Amount_Term.isnull(),'Loan_Amount_Term']=360
train.loc[train.Self_Employed.isnull(),'Self_Employed']='No'


In [21]:
pd.value_counts(train.Dependents)

0     360
1     102
2     101
3+     51
Name: Dependents, dtype: int64

### remove NAN values in test set

In [55]:
for col in test.columns:
    
    print(col,sum(test[col].isnull()))

Loan_ID 0
Gender 11
Married 0
Dependents 10
Education 0
Self_Employed 23
ApplicantIncome 0
CoapplicantIncome 0
LoanAmount 5
Loan_Amount_Term 6
Credit_History 29
Property_Area 0


In [58]:
train.columns

Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')

In [57]:
# prediction with sklearn(RF)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [63]:
x_cols=['Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area']
X=train[x_cols]
y=train['Loan_Status']      
X_train,X_test,y_train,y_test=train_test_split(X,y ,test_size=0.33, random_state=42)

clf=RandomForestClassifier()

clf.fit(X_train,y_train)

ValueError: could not convert string to float: 'Semiurban'

KeyError: True